## Practice: Data pipeline Hands-on Lab

Goal: Understand ETL logic.

We would need first to extract data from a MongoDB database comtaining customer reviews.

In [ ]:
#pip install mongomock

## Connect to an in-memory MongoDB instance

---



In [ ]:
import mongomock

# The code below create a simulated MongoDB client and database
client = mongomock.MongoClient() #not a real server for practicality and IT restrictions (stored in memory)
db = client.my_mock_database # database creation
review_collection = db.reviews # collection creation (collections are like tables in SQL)

## Extract data from MongoDB


In [ ]:
JSONL_path = "product_reviews_sales.jsonl"

| **Column**      | **Type**           | **Description**                                                                | **Example**                                           |
| --------------- | ------------------ | ------------------------------------------------------------------------------ | ----------------------------------------------------- |
| `product_id`    | `int`              | Unique product ID                  | `3`                                                   |
| `customer_id`   | `str`              | Unique identifier for the customer posting the review or making the purchase   | `"CUST_1042"`                                         |
| `review_text`   | `str`              | Free-text product review written by a customer                                 | `"Great quality and fits perfectly! Will buy again."` |
| `sale_date`     | `str` (ISO format) | Date of sale or review submission (YYYY-MM-DD)                                 | `"2024-11-09"`                                        |
| `units_sold`    | `int`              | Quantity sold during this transaction or review context                        | `2`                                                   |

In [ ]:
import json

# Load and parse all lines from the JSONL file
with open(JSONL_path, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]
    print(data)

In [ ]:
#Insert all records into the collection using .insert_many()
result = review_collection.insert_many(data)

#count how many documents have been inserted
print(f"Inserted {len(result.inserted_ids)} documents into 'reviews' collection.")

In [ ]:
#Let's check columns from data
sample = review_collection.find_one()
print("Fields in one document:", list(sample.keys()))

In [ ]:
import pandas as pd

df_reviews = pd.DataFrame((review_collection.find({})))
df_reviews.head()

## Extract data from an API


In [ ]:
API_URL = "https://fakestoreapi.com/products"


The Fake Store API is a free, public REST API that provides mock e-commerce data for learning and testing.
A REST API (Representational State Transfer API) is a style of web API that lets clients interact with a server using HTTP requests.
Most REST APIs transfer data in JSON.

Fetch JSON data

In [ ]:
import requests #popular Python library for making HTTP requests (GET, POST, PUT, DELETE, etc.).

#We send a GET request to the API endpoint to retrieve product data in JSON format:

resp = requests.get(API_URL)   # simple GET, no auth
products_json = resp.json()    # list of product
print(len(products_json), products_json[0])  # quick peek (count + first item)


In [ ]:
#transform json data into a dataframe
products_df = pd.DataFrame(products_json)
products_df.head()


In [ ]:
# Flatten the nested 'rating' dictionary
rating_df = pd.json_normalize(products_df['rating'])

In [ ]:
rating_df.head()

In [ ]:
products_df= products_df.drop(columns=['rating']).join(rating_df)

In [ ]:
products_df.head()

### Data transformation: MERGE DATA


We are going to merge both dataframes/combined data from 2 different sources.

In [ ]:
combined_df = df_reviews.merge(
    products_df,
    left_on="product_id",
    right_on="id",
    how="left"         # keep all reviews, even if product missing
)
combined_df.head()


A left join means:

Keep all rows from the left DataFrame (df_reviews)

Only bring product info if a match exists

If no matching product exists → fill with NaN


In real-world ETL, not all data sources align perfectly.

Some reviews may refer to missing or discontinued products.

A left join preserves your operational data (reviews) while flagging missing lookups for investigation or logging.

Why NOT use a full outer join in this case?

A full outer join keeps everything from both tables, even when there’s no match:

All reviews (even if the product is missing)

All products (even if nobody reviewed them)

But in a reviews → products lookup scenario, this usually adds noise, not value.

Let's save our combined dataset:

In [ ]:
combined_df.to_csv("combined_df.csv")